# Initial Data Analysis for 3A

First, the necesary imports. 
If Matplotlib does not want to display plots, install a GUI backend such as pyqt5 (available over pip) or tkinter.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np
from statistics import median
from preprocessing.relief import relief
from scipy.stats import f_oneway, zscore
from imblearn.over_sampling import SMOTE

np.random.seed(1234)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 7] # Change the desired figure size here if it is needed

Replace the following paths with the paths of the data.csv and labels.csv file on your system.

In [ ]:
df = pd.read_csv("./data/raw_data/data.csv", header= None)
labels = pd.read_csv("./data/raw_data/labels.csv", header=None)

We extract the indices of the two present classes for later

In [ ]:
orig_indices_class_1 = labels.index[labels[0] == 1]
orig_indices_class_2 = labels.index[labels[0] == 2]    

Let's take a look at the ranges of the various features to see if they are comparable (relevant for kNN)

In [ ]:
ranges = [max(df[i]) - min(df[i]) for i in range(0, df.shape[1])]
print(f"Max range of a feature at feature {np.argmax(ranges)}, range: {max(ranges)}")
print(f"Min range of a feature at feature {np.argmin(ranges)}, range: {min(ranges)}")
print(f"Median range: {median(ranges)}")

In [ ]:
plt.plot(sorted(ranges, reverse=True))
plt.xlabel("Features")
plt.ylabel("Range in feature")
plt.title("Feature range from largest to smallest")
plt.show()

Next, we take a look at how the classes are balanced in the dataset.

In [ ]:
balance = labels[0].value_counts()
print(balance)
y_pos = np.arange(2)
plt.bar(y_pos, [balance[1], balance[2]], width=0.6)
plt.xticks(y_pos, ("Class 1", "Class 2"))
plt.ylabel("Data points")
plt.title("Number of data points for each class")
plt.show()

Unfortunately, it seems the dataset is quite unbalanced.

## PCA

Next, we'll try an exploratory PCA to see how the PCs relate to the explained variance.

In [ ]:
pca = PCA(n_components=df.shape[1])
pca.fit(df)
cum_explained = np.cumsum(pca.explained_variance_ratio_)
plt.plot(cum_explained)
plt.xlabel("# of Principal Components included")
plt.ylabel("Cumulative variance explained")
plt.title("Cumulative variance explained by PCs")
plt.show()

Neat, it seems like PCA works relatively well. How well?

In [ ]:
print("PCs needed to explain variance:")
print(f">=90% of variance explained at PC: {np.argmax(cum_explained>=0.9)}")
print(f">=95% of variance explained at PC: {np.argmax(cum_explained>=0.95)}")
print(f">=99% of variance explained at PC: {np.argmax(cum_explained >= 0.99)}")

Lets restrict the number of dimensions to 3 for visualization

In [ ]:
pca3 = PCA(n_components=3)
pca3.fit(df)
data_pca3 = pd.DataFrame(pca3.transform(df))

print(f"Variance explained by the first 3 PCs: {sum(pca3.explained_variance_ratio_)}")

plt.rcParams['figure.figsize'] = [14, 10]

fig = plt.figure()
ax = fig.add_subplot(projection="3d")

points_class_1 = data_pca3.iloc[orig_indices_class_1]
points_class_2 = data_pca3.iloc[orig_indices_class_2]

ax.scatter(points_class_1[0], points_class_1[1], points_class_1[2], color='b', marker='o', label="Class 1")
ax.scatter(points_class_2[0], points_class_2[1], points_class_2[2], color='r', marker='^', label="Class 2")

ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_zlabel("PC 3")
ax.set_title("Labelled data projected to the first 3 PCs")
plt.legend()
plt.show()

One last interesting visualisation is to see how the data from the unlabelled set is distributed over this space.

In [ ]:
unlabelled_data = data_pca3.iloc[labels.shape[0]:, :]

plt.rcParams['figure.figsize'] = [14, 10]
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.scatter(unlabelled_data[0], unlabelled_data[1], unlabelled_data[2], color='g', marker = 'x')

ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_zlabel("PC 2")
ax.set_title("Unlabelled data projected to the first 3 PCs")
plt.show()

## ANOVA

We will perform anova for each feature, and see which features differ the most between groups.

In [ ]:
def anova(data:pd.DataFrame, indices_class_1:list, indices_class_2:list):
    

    pvals = []

    dps_c1 = data.iloc[indices_class_1]
    dps_c2 = data.iloc[indices_class_2]

    for feature_idx in range(data.shape[1]):
        c1_feature = dps_c1.iloc[:, feature_idx]
        c2_feature = dps_c2.iloc[:, feature_idx]
    
        f = f_oneway(c1_feature, c2_feature)
        pvals.append(f.pvalue)

    feature_indices = list(range(df.shape[1]))
    
    return zip(*sorted(zip(pvals, feature_indices)))

sorted_pvals, sorted_feature_indices = anova(df, orig_indices_class_1, orig_indices_class_2)

Lets see how many of these features return a significant difference.

In [ ]:
num_significant = sum(v < 0.05 for v in sorted_pvals)
print(num_significant)

Quite a lot. What if we check for a lower p-value (*** level)?

In [ ]:
num_significant = sum(v < 0.001 for v in sorted_pvals)
print(num_significant)

Still quite a few. For visualisation, we will show the first 10 or so.

In [ ]:
data_best_anova_1 = []
data_best_anova_2 = []

dps_c1 = df.iloc[orig_indices_class_1]
dps_c2 = df.iloc[orig_indices_class_2]
feature_indices = list(range(df.shape[1]))

for i in range(10):
    data_best_anova_1.append(dps_c1[feature_indices[i]])
    data_best_anova_2.append(dps_c2[feature_indices[i]])

# We change the size due to all the data
plt.rcParams['figure.figsize'] = [17, 12]
    
c = "blue"
plt.boxplot(data_best_anova_1, positions=list(range(1,11)), widths=0.3, patch_artist=True, 
            boxprops=dict(facecolor=c, color=c), capprops=dict(color=c),
            whiskerprops=dict(color=c), flierprops=dict(color=c, markeredgecolor=c), medianprops=dict(color=c))

c = "red"
plt.boxplot(data_best_anova_2, positions=[0.5 + i for i in range(1, 11)], widths=0.3, patch_artist=True, 
            boxprops=dict(facecolor=c, color=c), capprops=dict(color=c),
            whiskerprops=dict(color=c), flierprops=dict(color=c, markeredgecolor=c), medianprops=dict(color=c))

y_pos = [0.25 + i for i in range(1, 11)]
plt.xticks(y_pos, [f"Feature {i}" for i in sorted_feature_indices[:10]])

plt.ylabel("Feature values")
plt.xlabel("From left to right, most signicant differences between class features")

#Custom legend as matplotlib does not support labelling the way I'd like
legend_els = [Patch(facecolor='b', edgecolor='b', label='Class 1'), 
                Patch(facecolor='r', edgecolor='r',label='Class 2')]


plt.legend(handles=legend_els, loc="best")

    
plt.show()

# T-SNE 

In [ ]:
def plot_tsne3d(data: pd.DataFrame, indices_class_1, indices_class_2, title: str):
    plt.rcParams['figure.figsize'] = [14, 10]
    
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    
    points_class_1 = data.iloc[indices_class_1]
    points_class_2 = data.iloc[indices_class_2]
    
    ax.scatter(points_class_1[0], points_class_1[1], points_class_1[2], color='b', marker='o', label="Class 1")
    ax.scatter(points_class_2[0], points_class_2[1], points_class_2[2], color='r', marker='^', label="Class 2")
    
    ax.set_xlabel("t-SNE dimension 1")
    ax.set_ylabel("t-SNE dimension 2")
    ax.set_zlabel("t-SNE dimension 3")
    ax.set_title(title)
    plt.legend()
    plt.show()
    
tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df))
plot_tsne3d(tsne, orig_indices_class_1, orig_indices_class_2, "Labelled data projected to the 3D t-SNE space")


It seems that T-SNE can separate these points relatively well, in two islands.

# Impact of pre-processing methods

In this section we will evaluate the effect of various methods of pre-processing on the ability to separate the data using t-SNE. 

## Pre-processing without upsampling

### PCA with 9 leading eigenvectors ($\geq$95% of variance explained)

In [ ]:
pca9 = PCA(n_components=9)
data_pca9 = pd.DataFrame(pca9.fit_transform(df))

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(data_pca9))
plot_tsne3d(tsne, orig_indices_class_1, orig_indices_class_2, "Labelled data projected to the 3D t-SNE space after a PCA-transform")

It seems like the separation is slightly better, the red points have been projected much more outward.

### Z-Transformed data

In [ ]:
df2 = df.copy()
df2.apply(zscore)

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, orig_indices_class_1, orig_indices_class_2, "Labelled data projected to the 3D t-SNE space after a z-transform")

### Feature selection using ANOVA
We have previously found which features differ the most significantly according to ANOVA. So perhaps we can only use those for t-SNE? As above, we'll use the features whose difference in means was at least of a significance level ***.

In [ ]:
df2 = df.iloc[:,list(sorted_feature_indices[:73])]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, orig_indices_class_1, orig_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using ANOVA")

### Feature selection using Relief
We use a self-written relief function to perform the relief algorithm and then we analyse the results.
Minor precaution: This makes your computer work somewhat, so don't worry if it does not finish for a while.

In [ ]:
w = relief(df, labels, labels.shape[0])
print(f"Most relevant weight: {max(w)}")
print(f"Median weight: {median(w)}")

sorted_weights, sorted_feature_weight_indices = zip(*sorted(zip(w, list(range(df.shape[1]))), reverse=True))

comp = pd.DataFrame()
comp['most_relevant_features_relief'] = sorted_feature_weight_indices[:10]
comp['most_relevant_features_ANOVA'] = sorted_feature_indices[:10]

comp

Interestingly, there are only 2 features in common between the two methods in the top 10. Let's try t-SNE using only these dimensions.

In [ ]:
df2 = df.iloc[:,list(sorted_feature_weight_indices[:10])]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, orig_indices_class_1, orig_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using Relief")

What if we combine these top 10s?

In [ ]:
indices = list(set(sorted_feature_weight_indices[:10]).union(set(sorted_feature_indices[:73])))
df2 = df.iloc[:,indices]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, orig_indices_class_1, orig_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using Relief and ANOVA")

## Pre-processing after upsampling
We will balance our data using SMOTE

In [ ]:
oversample = SMOTE()
df_upsampled, labels_upsampled = oversample.fit_resample(df.iloc[:labels.shape[0],:], labels)

upsample_indices_class_1 = labels_upsampled.index[labels_upsampled[0] == 1]
upsample_indices_class_2 = labels_upsampled.index[labels_upsampled[0] == 2]

### Base-line: Upsampling without further processing

In [ ]:
tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df_upsampled))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after a PCA-transform")

### PCA with 9 leading eigenvectors ($\geq$95% of variance explained)

In [ ]:
pca9 = PCA(n_components=9)
data_pca9 = pd.DataFrame(pca9.fit_transform(df_upsampled))

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(data_pca9))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after a PCA-transform")

### Z-transform

In [ ]:
df2 = df_upsampled.copy()
df2.apply(zscore)

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after a z-transform")

### ANOVA: based on previous selected features

In [ ]:
df2 = df_upsampled.iloc[:,list(sorted_feature_indices[:73])]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using ANOVA")

### ANOVA: Applied to the new dataset

In [ ]:
sorted_pvals_up, sorted_feature_indices_up = anova(df_upsampled, upsample_indices_class_1, upsample_indices_class_2)

num_significant = sum(v < 0.001 for v in sorted_pvals_up)
print(num_significant)

In [ ]:
df2 = df_upsampled.iloc[:,list(sorted_feature_indices[:num_significant])]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using ANOVA")

It might be interesting to see how the top 10 best features differ between ANOVA prior to upsampling and after.

In [ ]:
comp = pd.DataFrame()
comp['most_relevant_features_anova'] = sorted_feature_indices[:10]
comp['most_relevant_features_anova_up'] = sorted_feature_indices_up[:10]

comp

Similar for half of the list. 

### Relief

Note: This will take even longer than before as the time complexity of relief is unfortunately quadratic if we use all points.

In [ ]:
w = relief(df_upsampled, labels_upsampled, labels_upsampled.shape[0])
print(f"Most relevant weight: {max(w)}")
print(f"Median weight: {median(w)}")

sorted_weights_up, sorted_feature_weight_indices_up = zip(*sorted(zip(w, list(range(df.shape[1]))), reverse=True))

comp = pd.DataFrame()
comp['most_relevant_features_relief'] = sorted_feature_weight_indices[:10]
comp['most_relevant_features_relief_up'] = sorted_feature_weight_indices_up[:10]

comp

4 matches between the relief approach prior to upsampling, and after upsampling.

In [ ]:
df2 = df_upsampled.iloc[:,list(sorted_feature_weight_indices_up[:10])]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using Relief")

And now, a combination again between the features from ANOVA and Relief:

In [ ]:
indices = list(set(sorted_feature_weight_indices_up[:10]).union(set(sorted_feature_indices_up[:73])))
print(len(indices))
df2 = df_upsampled.iloc[:,indices]

tsne = pd.DataFrame(TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(df2))
plot_tsne3d(tsne, upsample_indices_class_1, upsample_indices_class_2, "Labelled data projected to the 3D t-SNE space after feature selection using Relief and ANOVA")